In [1]:
from squad_solver import *
from utils_squads import *
from experiments_runner import run_experiment, repeat_run_experiment

params = {
    "budget": 400000,
    "number_of_defenders": 4,
    "number_of_midfielders": 3,
    "number_of_forwards": 3,
    "min_chemistry": 15,
    "M":11
}

general_components = "Avg_Global, PositionGK, PositionDEF, PositionMID, PositionFOR, Budget"
general_components += ", y, natCount, chemistryNat1, chemistryNat2, chemistryNat3"
formulation_1 = "problem Formulation_1: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R4;"
formulation_2 = "problem Formulation_2: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R34A, R34B, R34C, R4;"
formulation_3 = "problem Formulation_3: " + general_components + ", " + "natCountAll, R35, R36, R37, R38, R39, R33A, R33B, R33C, R4;"
formulation_4 = "problem Formulation_4: " + general_components + ", " + "natCountAll, R35, R36, R37, R38, R39, R33A, R33B, R33C, R34A, R34B, R34C, R4;"
formulation_5 = "problem Formulation_5: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R4B;"


def get_solved_elased_time_str(results):
    times = [r["solve_elapsed_time"] for r in results]
    median = np.median(times)
    std = np.std(times)
    return f"{median:.3f} ± ({std:.3f}) seconds"

In [3]:
results = {}; squads = {}

formulation_name = "Formulation_1"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)
    
formulation_name = "Formulation_2"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10)

formulation_name = "Formulation_3"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10)

formulation_name = "Formulation_4"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_4", formulation_4, print_results=False, print_summary=False, n_repeats=10)

formulation_name = "Formulation_5"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_5", formulation_5, print_results=False, print_summary=False, n_repeats=10)

In [5]:
def get_solved_elased_time_str(results):
    times = [r["solve_elapsed_time"] for r in results]
    median = np.median(times)
    std = np.std(times)
    return f"{median:.3f} ± ({std:.3f}) seconds"

new_times = {}
for key in results.keys():
    new_times[key] = get_solved_elased_time_str(results[key])


summary_results = pd.DataFrame([results[key][0] for key in results.keys()], index=results.keys()).drop("output", axis=1)
summary_results["solve_elapsed_time"] = summary_results.index.map(new_times)
display(summary_results)
summary_results.to_latex("table_formulaciones.tex", float_format="%.3f", escape=False)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation_1,solved,8580,8401,0.0,12524,1,0.516 ± (0.040) seconds,90.363636
Formulation_2,solved,8580,13438,0.0,14434,37,1.242 ± (0.056) seconds,90.363636
Formulation_3,solved,10259,13438,0.0,23894,39,1.852 ± (0.108) seconds,90.363636
Formulation_4,solved,10259,18475,0.0,29119,1,2.218 ± (0.078) seconds,90.363636
Formulation_5,solved,8580,8401,0.0,12645,1,0.688 ± (0.051) seconds,90.363636


### Try another value of M

In [8]:
results_M = {}; squads_M = {}

params["M"] = 11
formulation_name = "Formulation_1_M11"
results_M[formulation_name], squads_M[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)

params["M"] = 1867
formulation_name = "Formulation_1_M1867"
results_M[formulation_name], squads_M[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)

params["M"] = 10^6
formulation_name = "Formulation_1_M10^6"
results_M[formulation_name], squads_M[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)

In [9]:
new_times = {}
for key in results_M.keys():
    new_times[key] = get_solved_elased_time_str(results_M[key])


summary_results = pd.DataFrame([results_M[key][0] for key in results_M.keys()], index=results_M.keys()).drop("output", axis=1)
summary_results["solve_elapsed_time"] = summary_results.index.map(new_times)
display(summary_results)
summary_results.to_latex("table_M.tex", float_format="%.3f", escape=False)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation_1_M11,solved,8580,8401,0.0,12524,1,0.469 ± (0.031) seconds,90.363636
Formulation_1_M1867,solved,8580,8401,0.0,16941,1,0.796 ± (0.014) seconds,90.363636
Formulation_1_M10^6,solved,8580,8401,0.0,15102,1,0.571 ± (0.017) seconds,90.363636


### AMPL Gurobi cuts

In [18]:
results_cuts = {}; squads_cuts = {}
params["M"] = 11

formulation_name = "Formulation_1_cuts1"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10, cuts_param=1)
formulation_name = "Formulation_1_cuts2"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10, cuts_param=2)
formulation_name = "Formulation_1_cuts3"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10, cuts_param=3)

formulation_name = "Formulation_2_cuts1"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10, cuts_param=1)
formulation_name = "Formulation_2_cuts2"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10, cuts_param=2)
formulation_name = "Formulation_2_cuts3"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10, cuts_param=3)

formulation_name = "Formulation_3_cuts1"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10, cuts_param=1)
formulation_name = "Formulation_3_cuts2"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10, cuts_param=2)
formulation_name = "Formulation_3_cuts3"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10, cuts_param=3)

new_times = {}
for key in results_cuts.keys():
    new_times[key] = get_solved_elased_time_str(results_cuts[key])


summary_results = pd.DataFrame([results_cuts[key][0] for key in results_cuts.keys()], index=results_cuts.keys()).drop("output", axis=1)
summary_results["solve_elapsed_time"] = summary_results.index.map(new_times)
display(summary_results)
summary_results.to_latex("table_cuts.tex", float_format="%.3f", escape=False)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation_1_cuts1,solved,8580,8401,0.0,12524,1,0.485 ± (0.040) seconds,90.363636
Formulation_1_cuts2,solved,8580,8401,0.0,12524,1,0.484 ± (0.012) seconds,90.363636
Formulation_1_cuts3,solved,8580,8401,0.0,12524,1,0.625 ± (0.010) seconds,90.363636
Formulation_2_cuts1,solved,8580,13438,0.0,13983,1,0.937 ± (0.015) seconds,90.363636
Formulation_2_cuts2,solved,8580,13438,0.0,13975,1,0.946 ± (0.031) seconds,90.363636
Formulation_2_cuts3,solved,8580,13438,0.0,13971,1,1.157 ± (0.023) seconds,90.363636
Formulation_3_cuts1,solved,10259,13438,0.0,17613,1,1.969 ± (0.069) seconds,90.363636
Formulation_3_cuts2,solved,10259,13438,0.0,17605,1,2.352 ± (0.086) seconds,90.363636
Formulation_3_cuts3,solved,10259,13438,0.0,17547,1,2.046 ± (0.046) seconds,90.363636


In [14]:
_, _ = repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=True, print_summary=False, n_repeats=1, cuts_param=3)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
8401 constraints, all linear; 177799 nonzeros
	4 equality constraints
	8397 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 3
  cut:cuts = 3
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  3
InfUnbdInfo  1
Presolve  0

Optimize a model with 8401 rows, 8580 columns and 177799 nonzeros
Model fingerprint: 0x7915281b
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range    

In [15]:
_, _ = squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=True, print_summary=False, n_repeats=2, cuts_param=1)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 1
  cut:cuts = 1
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  1
InfUnbdInfo  1
Presolve  0

Optimize a model with 13438 rows, 8580 columns and 187873 nonzeros
Model fingerprint: 0x78bec64e
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range 

In [16]:
_, _ = squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=True, print_summary=False, n_repeats=2, cuts_param=2)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 2
  cut:cuts = 2
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  2
InfUnbdInfo  1
Presolve  0

Optimize a model with 13438 rows, 8580 columns and 187873 nonzeros
Model fingerprint: 0x78bec64e
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range 

In [17]:
_, _ = squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=True, print_summary=False, n_repeats=2, cuts_param=3)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 3
  cut:cuts = 3
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  3
InfUnbdInfo  1
Presolve  0

Optimize a model with 13438 rows, 8580 columns and 187873 nonzeros
Model fingerprint: 0x78bec64e
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range 